In [1]:
import os

import plotly.graph_objects as go
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

/Users/litto/miniconda/envs/TFG/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
VERSION = 3
MODEL_BASE_NAME = "MegaClassifier_a_EfficientNetB5"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [3]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 1
STEPS_PER_EPOCH = 200
IMAGE_SIZE = (456, 456)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [4]:
OPTIMIZERS = {"Adam": lambda x: tf.keras.optimizers.legacy.Adam(learning_rate=x)}

MIN_LEARNING_RATE = 1e-7
MAX_LEARNING_RATE = 1e-1

INITIAL_LEARNING_RATE = MIN_LEARNING_RATE

In [5]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

2025-03-08 18:47:53.668530: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-03-08 18:47:53.668555: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-08 18:47:53.668560: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-03-08 18:47:53.668589: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-08 18:47:53.668602: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
BATCH_SIZE = 16

In [7]:
LOSS_FUNCTIONS = {
    "BinaryFocalCrossentropy": lambda x, y: tf.keras.losses.BinaryFocalCrossentropy(
        alpha=x, gamma=y
    ),
    "SigmoidFocalCrossEntropy": lambda x, y: tfa.losses.SigmoidFocalCrossEntropy(
        alpha=x, gamma=y
    ),
}

ALPHAS = [
    0.25,
    0.6,
]

GAMMAS = [
    2.0,
    3.0,
]

In [8]:
class LRFinder(tf.keras.callbacks.Callback):
    def __init__(self, min_lr=1e-6, max_lr=1e-1, steps=100):
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.steps = steps
        self.lr_mult = (max_lr / min_lr) ** (1 / steps)
        self.lrs = []
        self.losses = []
        self.best_loss = np.inf

    def on_train_batch_end(self, batch, logs=None):
        logs = logs or {}
        loss = logs.get("loss")
        if loss is None:
            return

        self.lrs.append(tf.keras.backend.get_value(self.model.optimizer.lr))
        self.losses.append(loss)

        if loss < self.best_loss:
            self.best_loss = loss
        if loss > self.best_loss * 4:  # Detener si la pérdida explota
            self.model.stop_training = True

        # Aumentar el learning rate
        new_lr = self.lrs[-1] * self.lr_mult
        tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)

In [9]:
def smooth_curve(points, factor=0.4):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

In [ ]:
SUBVERSION = 0
ALPHA_SUBVERSION = 0
GAMMA_SUBVERSION = 0

for LOSS_FUNCTION in LOSS_FUNCTIONS:
    for ALPHA in ALPHAS:
        for GAMMA in GAMMAS:
            REPORTS_PATH = os.path.abspath(
                f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}"
            )

            train_datagen = ImageDataGenerator(
                preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
            )
            train_images = train_datagen.flow_from_dataframe(
                dataframe=train_dataset,
                x_col="file_name",
                y_col="binary_label",
                target_size=IMAGE_SIZE,
                batch_size=BATCH_SIZE,
                class_mode="binary",
                shuffle=True,
                seed=SEED,
            )

            pretrained_model = tf.keras.applications.EfficientNetB5(
                weights="imagenet",
                include_top=False,
                input_shape=IMAGE_SHAPE,
            )
            pretrained_model.trainable = False

            model = tf.keras.Sequential(
                [
                    pretrained_model,
                    tf.keras.layers.GlobalAveragePooling2D(),
                    tf.keras.layers.Dense(1, activation="sigmoid"),
                ],
                name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
            )

            model.compile(
                optimizer=OPTIMIZERS["Adam"](x=INITIAL_LEARNING_RATE),
                loss=LOSS_FUNCTIONS[LOSS_FUNCTION](x=ALPHA, y=GAMMA),
            )

            lr_finder = LRFinder(
                min_lr=MIN_LEARNING_RATE,
                max_lr=MAX_LEARNING_RATE,
                steps=STEPS_PER_EPOCH,
            )

            history = model.fit(
                train_images,
                epochs=EPOCHS,
                steps_per_epoch=STEPS_PER_EPOCH,
                callbacks=[
                    lr_finder,
                ],
            )

            smoothed_losses = smooth_curve(lr_finder.losses)

            fig = go.Figure()
            fig.add_trace(go.Scatter(x=lr_finder.lrs, y=smoothed_losses, mode="lines"))
            fig.update_layout(
                xaxis_type="log",
                title=f"Learning Rate Finder: {LOSS_FUNCTION} - α={ALPHA} - γ={GAMMA}",
                xaxis_title="Learning Rate",
                yaxis_title="Loss",
                template="seaborn",
                width=800,
                height=600,
                xaxis=dict(
                    type="log",
                    tickformat=".0e",
                    nticks=8,
                ),
            )
            fig.show()

            os.makedirs(REPORTS_PATH, exist_ok=True)
            fig.write_image(
                os.path.join(
                    REPORTS_PATH,
                    os.path.join(
                        REPORTS_PATH,
                        f"learning_rate_finder_v{VERSION}.{SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.png",
                    ),
                )
            )
            GAMMA_SUBVERSION += 1

        ALPHA_SUBVERSION += 1
        GAMMA_SUBVERSION = 0

    SUBVERSION += 1
    ALPHA_SUBVERSION = 0
    GAMMA_SUBVERSION = 0

Found 17906 validated image filenames belonging to 2 classes.


2025-03-08 18:48:00.462007: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - 199s 972ms/step - loss: 0.1486


Found 17906 validated image filenames belonging to 2 classes.


2025-03-08 18:51:23.609112: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

2025-03-08 19:08:57.123417: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - 219s 1s/step - loss: 0.0888


Found 17906 validated image filenames belonging to 2 classes.


2025-03-08 19:12:38.731816: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - 222s 1s/step - loss: 0.0436
